# Build Environment

In [ ]:
import numpy as np
import fileinput
import pickle

class build_environment(object):

  def __init__(self, K, threshold, T, trials):
    self.K = K + 1
    self.mu = None
    self.mu_t = None
    self.threshold = threshold
    self.true_epsilon_arms = []
    self.T = T
    self.trials = trials
    self.G = None
    self.sig = .1
    self.safe_arm = None

    self.baseline = self.K - 1


    
    
    self.best_arm = None

  def find_safe_arms(self, mu):

    safe_set = []
    for k in range(0,self.K-1):
      if mu[k] >= self.alpha*self.threshold:
        safe_set.append(k)
    
    return safe_set


  def readenv(self,p):
    data=[]
    filename="M44_S"+str(p)+".txt"
    for line in fileinput.input([filename]):

      try:
        line1 = [line.split(",") or line.split("\n") ]
        #print numpy.shape(line1)
        #print line1
        take=[]
        for i in range(len(line1[0])):
          take.append(float(line1[0][i]))
          #print take
          data.append(take)
      except ValueError as e:
        print(e)
    # print(data)
    means=(data[0])
    variance=(data[1])
    #print(len(self.means))
    # bestAction=max(range(self.K), key=lambda i: means[i])
    # bestVar=max(range(self.K), key=lambda i: variance[i])
    #print(self.means)
    # print(bestAction,bestVar)
    return np.array(means)
  
  def build_env1(self,):

    self.changepoint_time = [int(self.T/4.0), int((2/4.0)*self.T), int((3/4.0)*self.T), self.T]
    self.G = len(self.changepoint_time)
    print( self.changepoint_time, self.G)

    #self.safe_arm = 2

    self.best_arm = np.zeros(self.G)
    self.mu = np.zeros((self.G, self.K))
    #self.time_change = np.zeros((self.G, self.K))
    self.mu_t = np.zeros((self.T,self.K))
    self.safe_arm_t = np.zeros((self.T,3))
    
    self.baseline = self.K - 1
    self.mu_baseline = 0.35
    
    # self.mu[0, :] = [0.2, 0.65, 0.5, 0.3, 0.8, self.mu_baseline]
    # print(len(self.readenv(0)))
    self.mu[0, 0:self.K - 1] = self.readenv(0)
    self.mu[0, self.K - 1] = self.mu_baseline

    self.best_arm[0] = np.argmax(self.mu[0])
    self.mu_t[0:self.changepoint_time[0],:] = self.mu[0,:]
    #self.safe_arm_t[0:self.changepoint_time[0],:] = tuple(self.find_safe_arms(self.mu[0, :]))

    # self.mu[1, :] = [0.7, 0.65, 0.5, 0.3, 0.25, self.mu_baseline] 
    self.mu[1, 0:self.K - 1] = self.readenv(1)
    self.mu[1, self.K - 1] = self.mu_baseline

    self.best_arm[1] = np.argmax(self.mu[1])
    self.mu_t[self.changepoint_time[0]:self.changepoint_time[1],:] = self.mu[1,:]
    #self.safe_arm_t[self.changepoint_time[0]:self.changepoint_time[1],:] = (self.find_safe_arms(self.mu[1, :]))

    # self.mu[2, :] = [0.1, 0.65, 0.5, 0.8, 0.25, self.mu_baseline] 
    self.mu[2, 0:self.K - 1] = self.readenv(2)
    self.mu[2, self.K - 1] = self.mu_baseline

    self.best_arm[2] = np.argmax(self.mu[2])
    self.mu_t[self.changepoint_time[1]:self.changepoint_time[2],:] = self.mu[2,:]
    #self.safe_arm_t[self.changepoint_time[1]:self.changepoint_time[2],:] = (self.find_safe_arms(self.mu[2, :]))

    # self.mu[3, :] = [0.65, 0.65, 0.5, 0.3, 0.75, self.mu_baseline] 
    self.mu[3, 0:self.K - 1] = self.readenv(3)
    self.mu[3, self.K - 1] = self.mu_baseline

    self.best_arm[3] = np.argmax(self.mu[3])
    self.mu_t[self.changepoint_time[2]:self.changepoint_time[3],:] = self.mu[3,:]
    #self.safe_arm_t[self.changepoint_time[2]:self.changepoint_time[3],:] = (self.find_safe_arms(self.mu[3, :]))

  def pull_arm(self, arm, t):

    return self.mu_t[t][arm] + self.sig*np.random.randn()
    
# obj = build_environment(30, 0.1, 1000, 10)
# obj.build_env1()
# print(obj.mu)
# print(obj.mu_t)
# print(obj.pull_arm(1, 1))

# Plot Environment

In [ ]:
import matplotlib
import matplotlib.pyplot as plt


env = build_environment(30, 0.4, 8000, 10)
env.build_env1()
print(env.true_epsilon_arms)

plt.style.use('ggplot')
plt.figure(dpi=200)
# plt.plot([i for i in range(0,8000)], [env.threshold for i in range(0,8000)], '--', color='b', label = "Threshold")
#plt.plot([i for i in range(0,8000)], [1.5*env.threshold for i in range(0,8000)], '--', color='b')
#plt.plot(env.mu, 'ro')
plt.plot([i for i in range(0,8000)], env.mu_t[0:8000, 0:env.K-1])
plt.plot([i for i in range(0,8000)], env.mu_t[0:8000, env.K-1], label = "Baseline")

plt.title('Local Changepoint Environment (Movielens)', size = 15, fontweight='bold')
plt.ylabel('Means', size = 15, fontweight='bold')
plt.xlabel('Rounds', size = 15, fontweight='bold')
    
plt.yticks(size = 15, fontweight='bold')
plt.xticks(size = 15, fontweight='bold')

#plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
plt.ticklabel_format(axis='x', style='sci', scilimits=(0,0))



plt.legend()
plt.grid()
plt.show()

[2000, 4000, 6000, 8000] 4
[]


# Set Parameters

In [ ]:
T = 8000
trials = 4
env = build_environment(30, 0.4, T, trials)
env.build_env1()

[2000, 4000, 6000, 8000] 4


# Unbalanced MOSS

In [ ]:
import numpy as np

class UMOSS_Sampling(object):

  def __init__(self, env, confidence, type_):
    
    
    self.env = env
    self.confidence = confidence
    
    self.K = env.K
    self.T = env.T
    self.trials = env.trials


    self.mu_hat = np.zeros((self.T, self.K))
    self.sumreward = np.zeros((self.T, self.K))
    self.reward_arm = np.zeros((self.T, self.K))
    self.numplays = np.zeros((self.T, self.K))
    
    self.tol = 0.01
    self.gamma = 0.01
    
    self.budget = 0
    self.budget_error = 0
    self.alpha = 0.7

    self.estimated_G = np.zeros(self.K)
    self.type_ = type_
    
    
    
    self.error = np.zeros((self.trials, self.T))




  def run_algorithm(self, tr):

    self.seed = tr
    np.random.seed(self.seed)
    self.t = 0
    self.ts = np.zeros(self.K)
    self.tp = np.zeros(self.K)

    self.mu_hat = np.zeros((self.T, self.K))
    self.sumreward = np.zeros((self.T, self.K))
    self.reward_arm = np.zeros((self.T, self.K))
    self.numplays = np.zeros((self.T, self.K))

    #Sample each arm once
    

    for i in range(0,self.K):
      
      arm = i
      reward = self.env.pull_arm(arm, self.t)
      #self.sumreward[self.t][arm] += reward
      self.reward_arm[self.t][arm] = reward
      self.numplays[self.t][arm] = 1
      #self.mu_hat[self.t][arm] = self.sumreward[self.t][arm]/self.numplays[self.t][arm]

      self.tp[arm] += 1

      self.t += 1



    sum1 = 0.0
    self.ucb = np.zeros(self.K)
    self.lcb = np.zeros(self.K)
    self.B = np.zeros(self.K)
    while self.t < self.T:
      
      for k in range(self.K):

        if k == self.env.baseline:
          self.B[k] = (self.T*self.K)/(np.sqrt(self.T*self.K) + (self.K/(self.alpha*self.env.mu_t[self.t][self.env.baseline])))
        else:
          self.B[k] = np.sqrt(self.T*self.K) + (self.K/(self.alpha*self.env.mu_t[self.t][self.env.baseline]))
        
        n_k = (self.B[k]*self.B[k])/(self.T*self.T)
        beta_ucb = np.sqrt(4.0*np.log(max(0, (n_k/ (np.sum(self.numplays[int(self.ts[k]):self.t, k]) + 1) ) ))/(np.sum(self.numplays[int(self.ts[k]):self.t, k]) + 1)) - np.sqrt(1.0/n_k)
        self.ucb[k] = np.sum(self.reward_arm[int(self.ts[k]):self.t, k])/(np.sum(self.numplays[int(self.ts[k]):self.t, k]) + 1) + beta_ucb
        self.lcb[k] = max(0.0,np.sum(self.reward_arm[int(self.ts[k]):self.t, k])/(np.sum(self.numplays[int(self.ts[k]):self.t, k]) + 1) - beta_ucb)

      arm = np.argmax(self.ucb)

      reward = self.env.pull_arm(arm, self.t)
      #self.sumreward[self.t][arm] += reward
      self.reward_arm[self.t][arm] = reward
      self.numplays[self.t][arm] = 1
      #self.mu_hat[self.t][arm] = self.sumreward[self.t][arm]/self.numplays[self.t][arm]
      self.mu_hat[self.t, :] = [np.sum(self.reward_arm[int(self.ts[i]):self.t,i])/(np.sum(self.numplays[int(self.ts[i]):self.t,i]) + 1 ) for i in range(self.K)]
      
      self.tp[arm] += 1
      #print("t:"+str(self.numplays[self.t]))
      # if self.t > 1.0*self.K:
      #   #print(t)

      self.budget +=  self.lcb[arm] + self.ucb[np.argmax(self.ucb)] - (1.0-self.alpha)*self.ucb[self.env.baseline]
      #self.budget_error += self.env.mu_t[self.t][arm] + self.env.mu_t[self.t][np.argmax(self.env.mu_t[self.t])] - (1.0 - self.alpha)*self.env.mu_t[self.t][self.env.safe_arm]
      #self.budget_error += (1.0 - self.alpha)*self.env.mu_t[self.t][np.argmin(self.env.safe_arm_t[self.t])] - self.env.mu_t[self.t][arm] - self.env.mu_t[self.t][np.argmin(self.env.mu_t[self.t])]
      

      error1 = 0
      
      #max(-1.0*self.budget_error,0.0)
      error1 = self.env.mu_t[self.t][np.argmax(self.env.mu_t[self.t])] - self.env.mu_t[self.t][arm] + max(self.budget_error, 0.0)
      #print(self.env.mu_t[self.t][np.argmax(self.env.mu_t[self.t])], self.env.mu_t[self.t][arm])
      sum1 += error1
      self.error[tr][self.t] =  sum1
      #print(self.t,sum1)   
      
      if self.t%100 == 0 and self.t > 2.0*self.K:
        print("t " +str(self.t))
        print("t reward-arm: " + str([np.sum(self.reward_arm[int(self.ts[i]):self.t,i]) for i in range(self.K)]   ))
        #print(np.sum(self.tp), [int(self.tp[i]) for i in range(self.K)])
        #print(np.sum(self.numplays[int(np.min(self.ts)):int(np.max(self.tp)),:], axis = 1))
        #print(self.numplays[int(self.ts[2]):int(self.tp[2]),2], np.shape(self.numplays[int(self.ts[2]):int(self.tp[2]),2]))
        #print(self.numplays[int(self.ts[2]):self.t,2], np.sum(self.numplays[int(self.ts[2]):self.t,2]))
        print("t numplay: " + str([np.sum(self.numplays[int(self.ts[i]):self.t,i]) for i in range(self.K)]   ), str(np.sum(self.numplays[int(self.ts[i]):self.t,:])))
        print("t mu-hat: " + str([np.sum(self.reward_arm[int(self.ts[i]):self.t,i])/(np.sum(self.numplays[int(self.ts[i]):self.t,i]) + 1 ) for i in range(self.K)]   ))
        print("T true mu: " + str(self.env.mu_t[self.t]) + " Err: " + str(self.error[tr][self.t]))
        print(np.argmax(self.mu_hat[self.t]), np.argmax(self.env.mu_t[self.t]))
        print()
      
      self.t += 1

  def run_UMOSS_sampling(self, ):

    for tr in range(0,self.trials):
      
      print(tr, end=", ")
      self.run_algorithm(tr)


# T = 8000
# trials = 10
# env = build_environment(5, 0.4, T, trials)
# env.build_env1()
umoss = UMOSS_Sampling(env, 0.10, "global")
umoss.run_UMOSS_sampling()




0, t 100
t reward-arm: [35.690421305582404, 0.2800157208367223, 0.617873798410574, 0.7340893199201458, 0.7047557990149967, 0.3952722120123589, 0.597008841752559, 0.4728642791702302, 0.5006781148206442, 0.5360598501938372, 0.5044043571160878, 0.1454273506962975, 0.08610377251469935, 0.4921675016492828, 0.5233863232745426, 0.4753674327374267, 0.6214079073157606, 0.40748417362341993, 0.39430677016509014, 0.3535904260698275, 0.20970101841659217, 0.11136185954403606, 0.11544361988595057, 0.37178349795935584, 0.6789754623987607, 0.2915634325401235, 0.4365758517301446, 0.3922816149974166, 0.5192779214358457, 0.5589358769900284, 0.3654947425696916]
t numplay: [70.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0] 100.0
t mu-hat: [0.5026819902194705, 0.14000786041836116, 0.308936899205287, 0.3670446599600729, 0.35237789950749837, 0.19763610600617945, 0.2985044208762795, 0.2364321395851151, 0.25

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:83: RuntimeWarning: invalid value encountered in sqrt



t reward-arm: [88.31055101306077, 0.2800157208367223, 0.617873798410574, 0.7340893199201458, 0.7047557990149967, 0.3952722120123589, 0.597008841752559, 0.4728642791702302, 0.5006781148206442, 0.5360598501938372, 0.5044043571160878, 0.1454273506962975, 0.08610377251469935, 0.4921675016492828, 0.5233863232745426, 0.4753674327374267, 0.6214079073157606, 0.40748417362341993, 0.39430677016509014, 0.3535904260698275, 0.20970101841659217, 0.11136185954403606, 0.11544361988595057, 0.37178349795935584, 0.6789754623987607, 0.2915634325401235, 0.4365758517301446, 0.3922816149974166, 0.5192779214358457, 0.5589358769900284, 0.3654947425696916]
t numplay: [170.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0] 200.0
t mu-hat: [0.5164359708366127, 0.14000786041836116, 0.308936899205287, 0.3670446599600729, 0.35237789950749837, 0.19763610600617945, 0.2985044208762795, 0.2364321395851151, 0.2503390574

# DUCB

In [ ]:
import numpy as np

class DUCB_Sampling(object):

  def __init__(self, env, confidence, type_):
    
    
    self.env = env
    self.confidence = confidence
    
    self.K = env.K
    self.T = env.T
    self.trials = env.trials


    self.mu_hat = np.zeros((self.T, self.K))
    self.sumreward = np.zeros((self.T, self.K))
    self.reward_arm = np.zeros((self.T, self.K))
    self.numplays = np.zeros((self.T, self.K))
    
    self.tol = 0.01
    self.gamma = 0.01
    
    self.budget = 0
    self.budget_error = 0
    self.alpha = 0.7

    self.estimated_G = np.zeros(self.K)
    self.type_ = type_
    
    
    
    self.error = np.zeros((self.trials, self.T))




  def run_algorithm(self, tr):

    self.seed = tr
    np.random.seed(self.seed)
    self.t = 0
    self.ts = np.zeros(self.K)
    self.tp = np.zeros(self.K)

    self.mu_hat = np.zeros((self.T, self.K))
    self.sumreward = np.zeros((self.T, self.K))
    self.reward_arm = np.zeros((self.T, self.K))
    self.numplays = np.zeros((self.T, self.K))
    
    #Sample each arm once
    

    for i in range(0,self.K):
      
      arm = i
      reward = self.env.pull_arm(arm, self.t)
      #self.sumreward[self.t][arm] += reward
      self.reward_arm[self.t][arm] = reward
      self.numplays[self.t][arm] = 1
      #self.mu_hat[self.t][arm] = self.sumreward[self.t][arm]/self.numplays[self.t][arm]

      self.tp[arm] += 1

      self.t += 1



    sum1 = 0.0
    self.ucb = np.zeros(self.K)
    self.lcb = np.zeros(self.K)
    self.B = np.zeros(self.K)

    self.gamma = 1 - (1/(4.0*np.sqrt(self.T)))
    while self.t < self.T:
      
      for k in range(self.K):

        
        beta_ucb = 2.0*np.sqrt(2.0*np.log(self.t+1.0)/(np.sum(self.numplays[int(self.ts[k]):self.t, k]) + 1))
        #print(beta_ucb)
        self.ucb[k] = np.sum(self.reward_arm[int(self.ts[k]):self.t, k])/(np.sum(self.numplays[int(self.ts[k]):self.t, k]) + 1) + beta_ucb
        self.lcb[k] = max(0.0,np.sum(self.reward_arm[int(self.ts[k]):self.t, k])/(np.sum(self.numplays[int(self.ts[k]):self.t, k]) + 1) - beta_ucb)

      arm = np.argmax(self.ucb)

      reward = self.gamma*self.env.pull_arm(arm, self.t)
      #self.sumreward[self.t][arm] += reward
      self.reward_arm[self.t][arm] = reward
      self.numplays[self.t][arm] = self.gamma*1
      #self.mu_hat[self.t][arm] = self.sumreward[self.t][arm]/self.numplays[self.t][arm]
      self.mu_hat[self.t, :] = [np.sum(self.reward_arm[int(self.ts[i]):self.t,i])/(np.sum(self.numplays[int(self.ts[i]):self.t,i]) + 1 ) for i in range(self.K)]
      
      self.tp[arm] += 1
      #print("t:"+str(self.numplays[self.t]))
      # if self.t > 1.0*self.K:
      #   #print(t)

      self.budget +=  self.lcb[arm] + self.lcb[np.argmax(self.ucb)] - (1.0 - self.alpha)*self.ucb[np.argmin(self.env.safe_arm_t[self.t])]
      #self.budget_error += self.env.mu_t[self.t][arm] + self.env.mu_t[self.t][np.argmax(self.env.mu_t[self.t])] - (1.0 - self.alpha)*self.env.mu_t[self.t][self.env.safe_arm]
      #self.budget_error += (1.0 - self.alpha)*self.env.mu_t[self.t][np.argmin(self.env.safe_arm_t[self.t])] - self.env.mu_t[self.t][arm] - self.env.mu_t[self.t][np.argmin(self.env.mu_t[self.t])]
      

      error1 = 0
      
      #max(-1.0*self.budget_error,0.0)
      error1 = self.env.mu_t[self.t][np.argmax(self.env.mu_t[self.t])] - self.env.mu_t[self.t][arm] + max(self.budget_error, 0.0)
      #print(self.env.mu_t[self.t][np.argmax(self.env.mu_t[self.t])], self.env.mu_t[self.t][arm])
      sum1 += error1
      self.error[tr][self.t] =  sum1
      #print(self.t,sum1)   
      
      if self.t%100 == 0 and self.t > 2.0*self.K:
        print("t " +str(self.t))
        print("t reward-arm: " + str([np.sum(self.reward_arm[int(self.ts[i]):self.t,i]) for i in range(self.K)]   ))
        #print(np.sum(self.tp), [int(self.tp[i]) for i in range(self.K)])
        #print(np.sum(self.numplays[int(np.min(self.ts)):int(np.max(self.tp)),:], axis = 1))
        #print(self.numplays[int(self.ts[2]):int(self.tp[2]),2], np.shape(self.numplays[int(self.ts[2]):int(self.tp[2]),2]))
        #print(self.numplays[int(self.ts[2]):self.t,2], np.sum(self.numplays[int(self.ts[2]):self.t,2]))
        print("t numplay: " + str([np.sum(self.numplays[int(self.ts[i]):self.t,i]) for i in range(self.K)]   ), str(np.sum(self.numplays[int(self.ts[i]):self.t,:])))
        print("t mu-hat: " + str([np.sum(self.reward_arm[int(self.ts[i]):self.t,i])/(np.sum(self.numplays[int(self.ts[i]):self.t,i]) + 1 ) for i in range(self.K)]   ))
        print("T true mu: " + str(self.env.mu_t[self.t]) + " Err: " + str(self.error[tr][self.t]))
        print(np.argmax(self.mu_hat[self.t]), np.argmax(self.env.mu_t[self.t]))
        print()
      
      self.t += 1

  def run_DUCB_sampling(self, ):

    for tr in range(0,self.trials):
      
      print(tr, end=", ")
      self.run_algorithm(tr)


# T = 8000
# trials = 10
# env = build_environment(5, 0.4, T, trials)
# env.build_env1()
ducb = DUCB_Sampling(env, 0.10, "local")
ducb.run_DUCB_sampling()




0, t 100
t reward-arm: [2.0918022724738554, 0.6585312324647175, 2.302140791063339, 2.2519730776080085, 1.4867654794747414, 1.270465373209042, 2.333785300128756, 1.2901387355483713, 2.2543195619625203, 2.07212437561304, 1.3171159073305005, 0.20407526299706463, 0.09096233600661094, 1.2755240162904975, 1.4142681210385382, 1.2449219940962166, 2.096237204247473, 1.289211894762317, 1.1465296510784573, 1.1101366810056064, 1.3156734759465845, 0.09584162204343549, 0.24225922207260353, 1.256788630148234, 1.4552784962175873, 1.295664852968141, 1.388554857992892, 1.1928429893241812, 0.9544320571344788, 1.4556299026503614, 0.9278627683438827]
t numplay: [3.9916147450843757, 2.9944098300562505, 3.9916147450843757, 3.9916147450843757, 2.9944098300562505, 2.9944098300562505, 3.9916147450843757, 2.9944098300562505, 3.9916147450843757, 3.9916147450843757, 2.9944098300562505, 2.9944098300562505, 2.9944098300562505, 2.9944098300562505, 2.9944098300562505, 2.9944098300562505, 3.9916147450843757, 2.99440983

# UCB-CPD

In [ ]:
import numpy as np
from multiprocessing import Process, Queue
import time

class UCB_Sampling(object):

  def __init__(self, env, confidence, type_):
    
    
    self.env = env
    self.confidence = confidence
    
    self.K = env.K
    self.T = env.T
    self.trials = env.trials


    self.mu_hat = np.zeros((self.T, self.K))
    self.sumreward = np.zeros((self.T, self.K))
    self.reward_arm = np.zeros((self.T, self.K))
    self.numplays = np.zeros((self.T, self.K))
    
    self.tol = 0.01
    self.gamma = 0.01
    
    self.budget = 0
    self.budget_error = 0
    self.alpha = 0.7

    self.estimated_G = np.zeros(self.K)
    self.type_ = type_
    
    
    
    self.error = np.zeros((self.trials, self.T))
  

  def detect_changepoint(self, ts, tp):

   

    for k in range(self.K):
      
      #print(tp)
      for t_ in range(int(ts[k])+1,self.t):

        self.confidence = 1.0/self.t
        beta_ts = np.sqrt(0.5*np.log(np.log(self.t+1)/self.confidence)/(np.sum(self.numplays[int(ts[k]):t_, k]) + 1))
        beta_tp = np.sqrt(0.5*np.log(np.log(self.t+1)/self.confidence)/(np.sum(self.numplays[t_:self.t, k]) + 1))

        if (np.sum(self.reward_arm[int(ts[k]):t_, k])/(np.sum(self.numplays[int(ts[k]):t_, k])+1) - beta_ts > np.sum(self.reward_arm[t_:self.t, k])/(np.sum(self.numplays[t_:self.t, k]) + 1) + beta_tp) \
          or (np.sum(self.reward_arm[int(ts[k]):t_, k])/(np.sum(self.numplays[int(ts[k]):t_, k]) + 1) + beta_ts < np.sum(self.reward_arm[t_:self.t, k])/(np.sum(self.numplays[t_:self.t, k]) + 1) - beta_tp): 
            
         

          if self.type_ == "global":
            for k1 in range(self.K):

              self.estimated_G[k1] += 1

              #print(self.mu_hat[int(ts[k1]): int(tp[k1]), k1])
              self.mu_hat[int(ts[k1]): self.t, k1] = 0.0
              #print(self.mu_hat[int(ts[k1]): int(tp[k1]), k1])
              self.numplays[int(ts[k1]): self.t, k1] = 0
              self.sumreward[int(ts[k1]): self.t, k1] = 0.0
              self.reward_arm[int(ts[k1]): self.t, k1] = 0.0
              self.ts[k1] = self.t

              #print(self.reward_arm[0: self.t, k])
            print("Global Changepoint Detected at time " + str(self.t))
            
            break
          
          else:
            
            self.estimated_G[k] += 1
            self.mu_hat[int(ts[k]): self.t, k] = 0.0
            self.numplays[int(ts[k]): self.t, k] = 0
            self.sumreward[int(ts[k]): self.t, k] = 0.0
            self.reward_arm[int(ts[k]): self.t, k] = 0.0
            self.ts[k] = self.t

            

            print("Arm "+ str(k) +" Changepoint Detected at time " + str(self.t))
            continue



  def run_algorithm(self, tr):

    self.seed = tr
    np.random.seed(self.seed)
    self.t = 0
    self.ts = np.zeros(self.K)
    self.tp = np.zeros(self.K)

    self.mu_hat = np.zeros((self.T, self.K))
    self.sumreward = np.zeros((self.T, self.K))
    self.reward_arm = np.zeros((self.T, self.K))
    self.numplays = np.zeros((self.T, self.K))

    #Sample each arm once

    for i in range(0,self.K):
      
      arm = i
      reward = self.env.pull_arm(arm, self.t)
      #self.sumreward[self.t][arm] += reward
      self.reward_arm[self.t][arm] = reward
      self.numplays[self.t][arm] = 1
      #self.mu_hat[self.t][arm] = self.sumreward[self.t][arm]/self.numplays[self.t][arm]

      self.tp[arm] += 1

      self.t += 1



    sum1 = 0.0
    self.ucb = np.zeros(self.K)
    self.lcb = np.zeros(self.K)
    self.gamma = np.sqrt(np.log(self.T)/self.T)


    self.error_tr = np.zeros(self.T)

    while self.t < self.T:
      
      for k in range(self.K):

        beta_ucb = np.sqrt(1.0*np.log(np.log(self.t+1)/self.confidence)/(np.sum(self.numplays[int(self.ts[k]):self.t, k]) + 1))
        self.ucb[k] = np.sum(self.reward_arm[int(self.ts[k]):self.t, k])/(np.sum(self.numplays[int(self.ts[k]):self.t, k]) + 1) + beta_ucb
        self.lcb[k] = max(0.0,np.sum(self.reward_arm[int(self.ts[k]):self.t, k])/(np.sum(self.numplays[int(self.ts[k]):self.t, k]) + 1) - beta_ucb)

      if self.t% (self.K/self.gamma) <= self.K:
        arm = np.random.randint(0, self.K-1)
        #print("fe", arm, self.t% (self.K/self.gamma))
      else:
        arm = np.argmax(self.ucb)

      reward = self.env.pull_arm(arm, self.t)
      #self.sumreward[self.t][arm] += reward
      self.reward_arm[self.t][arm] = reward
      self.numplays[self.t][arm] = 1
      #self.mu_hat[self.t][arm] = self.sumreward[self.t][arm]/self.numplays[self.t][arm]
      self.mu_hat[self.t, :] = [np.sum(self.reward_arm[int(self.ts[i]):self.t,i])/(np.sum(self.numplays[int(self.ts[i]):self.t,i]) + 1 ) for i in range(self.K)]
      
      self.tp[arm] += 1
      #print("t:"+str(self.numplays[self.t]))
      # if self.t > 1.0*self.K:
      #   #print(t)

      self.budget +=  self.lcb[arm] + self.lcb[np.argmax(self.ucb)] - (1.0 - self.alpha)*self.ucb[np.argmin(self.env.safe_arm_t[self.t])]
      #self.budget_error += self.env.mu_t[self.t][arm] + self.env.mu_t[self.t][np.argmax(self.env.mu_t[self.t])] - (1.0 - self.alpha)*self.env.mu_t[self.t][self.env.safe_arm]
      #self.budget_error += (1.0 - self.alpha)*self.env.mu_t[self.t][np.argmin(self.env.safe_arm_t[self.t])] - self.env.mu_t[self.t][arm] - self.env.mu_t[self.t][np.argmin(self.env.mu_t[self.t])]


      if self.t%100 == 0:
        self.detect_changepoint(self.ts, self.tp)
      
      

      error1 = 0
      
      #max(-1.0*self.budget_error,0.0)
      error1 = self.env.mu_t[self.t][np.argmax(self.env.mu_t[self.t])] - self.env.mu_t[self.t][arm] + max(self.budget_error, 0.0)
      #print(self.env.mu_t[self.t][np.argmax(self.env.mu_t[self.t])], self.env.mu_t[self.t][arm])
      sum1 += error1
      # self.error[tr][self.t] =  sum1
      self.error_tr[self.t] =  sum1
      #print(self.t,sum1)   
      
      if self.t%100 == 0 and self.t > 2.0*self.K:
        print("t " +str(self.t))
        print("t reward-arm: " + str([np.sum(self.reward_arm[int(self.ts[i]):self.t,i]) for i in range(self.K)]   ))
        #print(np.sum(self.tp), [int(self.tp[i]) for i in range(self.K)])
        #print(np.sum(self.numplays[int(np.min(self.ts)):int(np.max(self.tp)),:], axis = 1))
        #print(self.numplays[int(self.ts[2]):int(self.tp[2]),2], np.shape(self.numplays[int(self.ts[2]):int(self.tp[2]),2]))
        #print(self.numplays[int(self.ts[2]):self.t,2], np.sum(self.numplays[int(self.ts[2]):self.t,2]))
        print("t numplay: " + str([np.sum(self.numplays[int(self.ts[i]):self.t,i]) for i in range(self.K)]   ), str(np.sum(self.numplays[int(self.ts[i]):self.t,:])))
        print("t mu-hat: " + str([np.sum(self.reward_arm[int(self.ts[i]):self.t,i])/(np.sum(self.numplays[int(self.ts[i]):self.t,i]) + 1 ) for i in range(self.K)]   ))
        print("T true mu: " + str(self.env.mu_t[self.t]) + " Err: " + str(self.error_tr[self.t]))
        print(np.argmax(self.mu_hat[self.t]), np.argmax(self.env.mu_t[self.t]))
        print(self.budget, self.budget_error)
        print()
      
      self.t += 1
    
    file = open("/content/f_UCBCPD_"+str(tr), 'wb')
    data = pickle.dump(self.error_tr,file)
    file.close()

  # def run_UCB_sampling(self, ):

  #   for tr in range(0,self.trials):
      
  #     print(tr, end=", ")
  #     self.run_algorithm(tr)
  
  def run_UCB_sampling(self):

        
      self.error = np.zeros((self.trials, self.T))  
      pool = []
      Q = Queue()

      for tr in range(self.trials):
        
        p = Process(target = self.run_algorithm, args=(tr, ))
        pool.append(p)
          
      for tr in range(self.trials):
        pool[tr].start()

      

      for tr in range(self.trials):
        # self.error[tr] = Q.get()
        pool[tr].join()
      
      for tr in range(self.trials):
        file = open("/content/f_UCBCPD_"+str(tr), 'rb')
        self.error[tr] = pickle.load(file)
        file.close()

      for tr in range(self.trials):
        pool[tr].close()


# T = 8000
# trials = 10
# env = build_environment(5, 0.4, T, trials)
# env.build_env1()
ucb = UCB_Sampling(env, 0.10, "local")
ucb.run_UCB_sampling()




t 100
t reward-arm: [1.9918075296478195, 1.038196765475988, 1.9799920656962011, 2.6552316063208203, 1.3594802503534098, 1.9602997764634493, 2.8715389788944092, 1.18560926086692, 2.1800232368948826, 1.3620207192239957, 1.3242014760225609, 0.24312246683371097, -0.05619048601908245, 2.0173875171379914, 1.224549028409988, 1.3666331386921822, 2.681089203257091, 1.3531338194443063, 0.5757958848423563, 1.8259039592992066, 2.064184266170492, 0.1265053768903041, 0.09413656668808656, 1.1897643346384903, 1.2440936784198344, 0.876130717707495, 1.1235055252641368, 0.5226662495330927, 0.9982437332467631, 1.2706925808190468, 1.3867248244328108]
t numplay: [4.0, 3.0, 4.0, 5.0, 3.0, 4.0, 5.0, 3.0, 4.0, 3.0, 3.0, 2.0, 2.0, 4.0, 3.0, 3.0, 5.0, 3.0, 2.0, 4.0, 4.0, 2.0, 2.0, 3.0, 3.0, 3.0, 3.0, 2.0, 3.0, 3.0, 3.0] 100.0
t mu-hat: [0.3983615059295639, 0.259549191368997, 0.3959984131392402, 0.44253860105347004, 0.33987006258835245, 0.3920599552926899, 0.4785898298157349, 0.29640231521673, 0.43600464737897654

# GLR-UCB

In [ ]:
import numpy as np
import math
from multiprocessing import Process, Queue
import time

class GLR_UCB_Sampling(object):

  def __init__(self, env, confidence, type_):
    
    
    self.env = env
    self.confidence = confidence
    
    self.K = env.K
    self.T = env.T
    self.trials = env.trials


    self.mu_hat = np.zeros((self.T, self.K))
    self.sumreward = np.zeros((self.T, self.K))
    self.reward_arm = np.zeros((self.T, self.K))
    self.numplays = np.zeros((self.T, self.K))
    
    self.tol = 0.01
    self.gamma = 0.01
    
    self.budget = 0
    self.budget_error = 0
    self.alpha = 0.7

    self.estimated_G = np.zeros(self.K)
    self.type_ = type_
    
    
    
    self.error = np.zeros((self.trials, self.T))
  
  def klGauss(self, x, y, sig2 = 0.1):
        """caution modified"""
        """Kullback-Leibler divergence for Gaussian distributions."""
        return ((x-y)*(x-y))/(2*sig2)

  def klucbGauss(self, x, d, sig2=1., precision=0.):
      """klUCB index computation for Gaussian distributions.
    
      Note that it does not require any search.
      """
      #print 2.0*sig2*d, x
      return x + math.sqrt(2.0*sig2*d)

  def klucb(self, x, d, upperbound, lowerbound=-float('inf'), precision=1e-6):
    
        """The generic klUCB index computation.
    
        Input args.: x, d, div, upperbound, lowerbound=-float('inf'), precision=1e-6,
        where div is the KL divergence to be used.
        """
        l = max(x, lowerbound)
        u = upperbound
        while u-l > precision:
            m = (l+u)/2
            if self.klGauss(x, m) > d:
                u = m
            else:
                l = m
        return (l+u)/2

  def detect_changepoint(self, ts, tp):

   

    for k in range(self.K):
      
      #print(tp)
      kl_list = []
      for t_ in range(int(ts[k])+1,self.t):

        # self.klucb(self.global_payoffSums[actionTaken]/(self.global_numPlays[actionTaken]+1), 
        #(math.log(self.t+1) + self.c * math.log(math.log(self.t+1)))  / (self.global_numPlays[actionTaken]+1), 1e-4) # Could adapt tolerance to the value of self.t


        
        # beta_ts = np.sqrt(0.5*np.log((self.t+1)/self.confidence)/(np.sum(self.numplays[int(ts[k]):t_, k]) + 1))
        # beta_tp = np.sqrt(0.5*np.log((self.t+1)/self.confidence)/(np.sum(self.numplays[t_:self.t, k]) + 1))

        # beta_ts = 5.0*np.log((self.t+1)/self.confidence)/(np.sum(self.numplays[int(ts[k]):t_, k]) + 1)
        # beta_tp = 5.0*np.log((self.t+1)/self.confidence)/(np.sum(self.numplays[t_:self.t, k]) + 1)

        emp_mean_ts = np.sum(self.reward_arm[int(ts[k]):t_, k])/(np.sum(self.numplays[int(ts[k]):t_, k])+1)
        emp_mean_tp = np.sum(self.reward_arm[t_:self.t, k])/(np.sum(self.numplays[t_:self.t, k]) + 1)
        emp_mean_tot = np.sum(self.reward_arm[int(ts[k]):self.t, k])/(np.sum(self.numplays[int(ts[k]):self.t, k]) + 1)

        s = np.sum(self.numplays[int(ts[k]):t_, k])+1
        n = (np.sum(self.numplays[t_:self.t, k]) + 1)
        n_s = n - s

        #kl_list.append(s*self.klucb(emp_mean_ts, emp_mean_tot, 1e-4) + n_s*self.klucb(emp_mean_tp, emp_mean_tot, 1e-4))
        kl_list.append(s*self.klGauss(emp_mean_ts, emp_mean_tot) + n_s*self.klGauss(emp_mean_tp, emp_mean_tot))
      
      self.confidence = 1.0/self.t
      self.conf_inf = 1.0*np.log(3.0*math.pow(self.t,3.0/2.0)/self.confidence)
      #self.conf_inf = 8.0*np.log(3.0*self.t)


      if max(kl_list) > self.conf_inf:
        
        #if s*self.klGauss(emp_mean_ts, emp_mean_tot) + n_s*self.klGauss(emp_mean_tp, emp_mean_tot) > self.conf_inf:
          if self.type_ == "global":
            for k1 in range(self.K):

              self.estimated_G[k1] += 1

              #print(self.mu_hat[int(ts[k1]): int(tp[k1]), k1])
              self.mu_hat[int(ts[k1]): self.t, k1] = 0.0
              #print(self.mu_hat[int(ts[k1]): int(tp[k1]), k1])
              self.numplays[int(ts[k1]): self.t, k1] = 0
              self.sumreward[int(ts[k1]): self.t, k1] = 0.0
              self.reward_arm[int(ts[k1]): self.t, k1] = 0.0
              self.ts[k1] = self.t

              #print(self.reward_arm[0: self.t, k])
            print("Global Changepoint Detected at time " + str(self.t))
            
            break
          
          else:
            
            self.estimated_G[k] += 1
            self.mu_hat[int(ts[k]): self.t, k] = 0.0
            self.numplays[int(ts[k]): self.t, k] = 0
            self.sumreward[int(ts[k]): self.t, k] = 0.0
            self.reward_arm[int(ts[k]): self.t, k] = 0.0
            self.ts[k] = self.t

            

            print("Arm "+ str(k) +" Changepoint Detected at time " + str(self.t))
            continue



  def run_algorithm(self, tr):

    self.seed = tr
    np.random.seed(self.seed)
    self.t = 0
    self.ts = np.zeros(self.K)
    self.tp = np.zeros(self.K)

    self.mu_hat = np.zeros((self.T, self.K))
    self.sumreward = np.zeros((self.T, self.K))
    self.reward_arm = np.zeros((self.T, self.K))
    self.numplays = np.zeros((self.T, self.K))
    
    #Sample each arm once
    

    for i in range(0,self.K):
      
      arm = i
      reward = self.env.pull_arm(arm, self.t)
      #self.sumreward[self.t][arm] += reward
      self.reward_arm[self.t][arm] = reward
      self.numplays[self.t][arm] = 1
      #self.mu_hat[self.t][arm] = self.sumreward[self.t][arm]/self.numplays[self.t][arm]

      self.tp[arm] += 1

      self.t += 1



    sum1 = 0.0
    self.ucb = np.zeros(self.K)
    self.lcb = np.zeros(self.K)
    self.gamma = np.sqrt(np.log(self.T)/self.T)

    self.error_tr = np.zeros(self.T)

    while self.t < self.T:
      
      for k in range(self.K):

        beta_ucb = np.sqrt(1.0*np.log(np.log(self.t+1)/self.confidence)/(np.sum(self.numplays[int(self.ts[k]):self.t, k]) + 1))
        self.ucb[k] = np.sum(self.reward_arm[int(self.ts[k]):self.t, k])/(np.sum(self.numplays[int(self.ts[k]):self.t, k]) + 1) + beta_ucb
        self.lcb[k] = max(0.0,np.sum(self.reward_arm[int(self.ts[k]):self.t, k])/(np.sum(self.numplays[int(self.ts[k]):self.t, k]) + 1) - beta_ucb)

      
      if self.t% (self.K/self.gamma) <= self.K:
        arm = np.random.randint(0, self.K-1)
        #print("fe", arm, self.t% (self.K/self.gamma))
      else:
        arm = np.argmax(self.ucb)

      reward = self.env.pull_arm(arm, self.t)
      #self.sumreward[self.t][arm] += reward
      self.reward_arm[self.t][arm] = reward
      self.numplays[self.t][arm] = 1
      #self.mu_hat[self.t][arm] = self.sumreward[self.t][arm]/self.numplays[self.t][arm]
      self.mu_hat[self.t, :] = [np.sum(self.reward_arm[int(self.ts[i]):self.t,i])/(np.sum(self.numplays[int(self.ts[i]):self.t,i]) + 1 ) for i in range(self.K)]
      
      self.tp[arm] += 1
      #print("t:"+str(self.numplays[self.t]))
      # if self.t > 1.0*self.K:
      #   #print(t)

      self.budget +=  self.lcb[arm] + self.lcb[np.argmax(self.ucb)] - (1.0 - self.alpha)*self.ucb[np.argmin(self.env.safe_arm_t[self.t])]
      #self.budget_error += self.env.mu_t[self.t][arm] + self.env.mu_t[self.t][np.argmax(self.env.mu_t[self.t])] - (1.0 - self.alpha)*self.env.mu_t[self.t][self.env.safe_arm]
      #self.budget_error += (1.0 - self.alpha)*self.env.mu_t[self.t][np.argmin(self.env.safe_arm_t[self.t])] - self.env.mu_t[self.t][arm] - self.env.mu_t[self.t][np.argmin(self.env.mu_t[self.t])]


      if self.t%100 == 0:
        self.detect_changepoint(self.ts, self.tp)
      
      

      error1 = 0
      
      #max(-1.0*self.budget_error,0.0)
      error1 = self.env.mu_t[self.t][np.argmax(self.env.mu_t[self.t])] - self.env.mu_t[self.t][arm] + max(self.budget_error, 0.0)
      #print(self.env.mu_t[self.t][np.argmax(self.env.mu_t[self.t])], self.env.mu_t[self.t][arm])
      sum1 += error1

      # self.error[tr][self.t] =  sum1
      self.error_tr[self.t] = sum1
      #print(self.t,sum1)   
      
      if self.t%100 == 0 and self.t > 2.0*self.K:
        print("t " +str(self.t))
        print("t reward-arm: " + str([np.sum(self.reward_arm[int(self.ts[i]):self.t,i]) for i in range(self.K)]   ))
        #print(np.sum(self.tp), [int(self.tp[i]) for i in range(self.K)])
        #print(np.sum(self.numplays[int(np.min(self.ts)):int(np.max(self.tp)),:], axis = 1))
        #print(self.numplays[int(self.ts[2]):int(self.tp[2]),2], np.shape(self.numplays[int(self.ts[2]):int(self.tp[2]),2]))
        #print(self.numplays[int(self.ts[2]):self.t,2], np.sum(self.numplays[int(self.ts[2]):self.t,2]))
        print("t numplay: " + str([np.sum(self.numplays[int(self.ts[i]):self.t,i]) for i in range(self.K)]   ), str(np.sum(self.numplays[int(self.ts[i]):self.t,:])))
        print("t mu-hat: " + str([np.sum(self.reward_arm[int(self.ts[i]):self.t,i])/(np.sum(self.numplays[int(self.ts[i]):self.t,i]) + 1 ) for i in range(self.K)]   ))
        print("T true mu: " + str(self.env.mu_t[self.t]) + " Err: " + str(self.error_tr[self.t]))
        print(np.argmax(self.mu_hat[self.t]), np.argmax(self.env.mu_t[self.t]))
        print(self.budget, self.budget_error)
        print()
      
      self.t += 1
    
    file = open("/content/f_GLRUCB_"+str(tr), 'wb')
    data = pickle.dump(self.error_tr,file)
    file.close()

  # def run_GLR_UCB_sampling(self, ):

  #   for tr in range(0,self.trials):
      
  #     print(tr, end=", ")
  #     self.run_algorithm(tr)
  
  def run_GLR_UCB_sampling(self):

        
      self.error = np.zeros((self.trials, self.T))  
      pool = []
      Q = Queue()

      for tr in range(self.trials):
        
        p = Process(target = self.run_algorithm, args=(tr, ))
        pool.append(p)
          
      for tr in range(self.trials):
        pool[tr].start()

      

      for tr in range(self.trials):
        # self.error[tr] = Q.get()
        pool[tr].join()
      
      for tr in range(self.trials):
        file = open("/content/f_GLRUCB_"+str(tr), 'rb')
        self.error[tr] = pickle.load(file)
        file.close()

      for tr in range(self.trials):
        pool[tr].close()


# T = 8000
# trials = 10
# env = build_environment(5, 0.4, T, trials)
# env.build_env1()
glrucb = GLR_UCB_Sampling(env, 0.10, "local")
glrucb.run_GLR_UCB_sampling()




t 100
t 100
t reward-arm: [2.1218184011366232, 0.5187812395014364, 2.280244718579417, 2.2101501835533686, 2.2164563728017055, 1.3342599047864132, 1.375377329035682, 1.301912517685055, 1.4281489514439125, 2.176409188249292, 2.193789096836124, 0.10340901172942468, -0.07586346534852716, 2.2571684410348882, 2.0309491343427784, 1.9930287353276355, 2.1389773272302253, 1.4328591369226322, 1.3695805450923126, 1.3965354146214826, 0.5311181778119329, 0.17676257868843387, 0.021380122965109785, 1.331337398761149, 2.0759897394911664, 1.0790544339765853, 1.9872153656516045, 1.0574526192085605, 1.208537480536901, 1.972565636132415, 0.9582074661451999]
t 100
t reward-arm: [2.9085674443657306, 0.40203827221398536, 2.820219957168995, 2.1427695602065446, 2.901398210452586, 1.1136609081528355, 2.8479308544093573, 2.0277738820227533, 2.1439011559430026, 1.4093000263913062, 2.1536458558622975, -0.20601407094976543, -0.10043703870133443, 1.3063307336311782, 1.9996351272605577, 1.3316018494761692, 1.760591410

# Conservative-UCB

In [ ]:
import numpy as np

class CUCB_Sampling(object):

  def __init__(self, env, confidence, alpha, type_):
    
    
    self.env = env
    self.confidence = confidence
    
    self.K = env.K
    self.T = env.T
    self.trials = env.trials


    self.mu_hat = np.zeros((self.T, self.K))
    self.sumreward = np.zeros((self.T, self.K))
    self.reward_arm = np.zeros((self.T, self.K))
    self.numplays = np.zeros((self.T, self.K))
    
    self.tol = 0.01
    self.gamma = 0.01
    
    self.budget = 0
    self.budget_error = 0
    self.alpha = 0.7

    self.estimated_G = np.zeros(self.K)
    self.type_ = type_
    
    
    
    self.error = np.zeros((self.trials, self.T))
  

  def detect_changepoint(self, ts, tp):

   

    for k in range(self.K):
      
      #print(tp)
      for t_ in range(int(ts[k])+1,self.t):

        
        beta_ts = np.sqrt(0.5*np.log((self.t+1)/self.confidence)/(np.sum(self.numplays[int(ts[k]):t_, k]) + 1))
        beta_tp = np.sqrt(0.5*np.log((self.t+1)/self.confidence)/(np.sum(self.numplays[t_:self.t, k]) + 1))

        if (np.sum(self.reward_arm[int(ts[k]):t_, k])/(np.sum(self.numplays[int(ts[k]):t_, k])+1) - beta_ts > np.sum(self.reward_arm[t_:self.t, k])/(np.sum(self.numplays[t_:self.t, k]) + 1) + beta_tp) \
          or (np.sum(self.reward_arm[int(ts[k]):t_, k])/(np.sum(self.numplays[int(ts[k]):t_, k]) + 1) + beta_ts < np.sum(self.reward_arm[t_:self.t, k])/(np.sum(self.numplays[t_:self.t, k]) + 1) - beta_tp): 
            
         

          if self.type_ == "global":
            for k1 in range(self.K):

              self.estimated_G[k1] += 1

              #print(self.mu_hat[int(ts[k1]): int(tp[k1]), k1])
              self.mu_hat[int(ts[k1]): self.t, k1] = 0.0
              #print(self.mu_hat[int(ts[k1]): int(tp[k1]), k1])
              self.numplays[int(ts[k1]): self.t, k1] = 0
              self.sumreward[int(ts[k1]): self.t, k1] = 0.0
              self.reward_arm[int(ts[k1]): self.t, k1] = 0.0
              self.ts[k1] = self.t

              #print(self.reward_arm[0: self.t, k])
            print("Global Changepoint Detected at time " + str(self.t))
            
            return True
          
          else:
            
            self.estimated_G[k] += 1
            self.mu_hat[int(ts[k]): self.t, k] = 0.0
            self.numplays[int(ts[k]): self.t, k] = 0
            self.sumreward[int(ts[k]): self.t, k] = 0.0
            self.reward_arm[int(ts[k]): self.t, k] = 0.0
            self.ts[k] = self.t

            

            print("Arm "+ str(k) +" Changepoint Detected at time " + str(self.t))
            continue



  def run_algorithm(self, tr):

    self.seed = tr
    np.random.seed(self.seed)
    self.t = 0
    self.ts = np.zeros(self.K)
    self.tp = np.zeros(self.K)

    self.budget = 0

    self.mu_hat = np.zeros((self.T, self.K))
    self.sumreward = np.zeros((self.T, self.K))
    self.reward_arm = np.zeros((self.T, self.K))
    self.numplays = np.zeros((self.T, self.K))

    #Sample each arm once
    

    for i in range(0,self.K):
      
      arm = i
      reward = self.env.pull_arm(arm, self.t)
      #self.sumreward[self.t][arm] += reward
      self.reward_arm[self.t][arm] = reward
      self.numplays[self.t][arm] = 1
      #self.mu_hat[self.t][arm] = self.sumreward[self.t][arm]/self.numplays[self.t][arm]

      self.tp[arm] += 1

      self.t += 1



    sum1 = 0.0
    self.ucb = np.zeros(self.K)
    self.lcb = np.zeros(self.K)

    while self.t < self.T:
      
      for k in range(self.K):

        beta_ucb = np.sqrt(1.5*np.log((self.t+1)/self.confidence)/(np.sum(self.numplays[int(self.ts[k]):self.t, k]) + 1))
        self.ucb[k] = np.sum(self.reward_arm[int(self.ts[k]):self.t, k])/(np.sum(self.numplays[int(self.ts[k]):self.t, k]) + 1) + beta_ucb
        self.lcb[k] = max(0.0,np.sum(self.reward_arm[int(self.ts[k]):self.t, k])/(np.sum(self.numplays[int(self.ts[k]):self.t, k]) + 1) - beta_ucb)

      if self.budget >= 0:
        arm = np.argmax(self.ucb)
      else:
        #arm = self.env.safe_arm
        arm = self.env.baseline

      reward = self.env.pull_arm(arm, self.t)
      #self.sumreward[self.t][arm] += reward
      self.reward_arm[self.t][arm] = reward
      self.numplays[self.t][arm] = 1
      #self.mu_hat[self.t][arm] = self.sumreward[self.t][arm]/self.numplays[self.t][arm]
      self.mu_hat[self.t, :] = [np.sum(self.reward_arm[int(self.ts[i]):self.t,i])/(np.sum(self.numplays[int(self.ts[i]):self.t,i]) + 1 ) for i in range(self.K)]
      
      self.tp[arm] += 1
      #print("t:"+str(self.numplays[self.t]))
      # if self.t > 1.0*self.K:
      #   #print(t)

      #self.budget +=  self.lcb[arm] + self.lcb[np.argmax(self.ucb)] - (1.0-self.alpha)*self.env.mu_t[self.t][np.argmin(self.env.safe_arm_t[self.t])]
      #print(self.env.baseline)
      #self.budget +=  self.lcb[arm] + self.lcb[np.argmax(self.ucb)] - (1.0-self.alpha)*self.ucb[self.env.baseline] - self.alpha*self.env.threshold
      #self.budget +=  self.lcb[arm] + self.lcb[np.argmax(self.ucb)] - (1.0-self.alpha)*self.ucb[self.env.baseline]
      self.budget +=  self.lcb[arm] + self.ucb[np.argmax(self.ucb)] - (1.0-self.alpha)*self.ucb[self.env.baseline]
      
      #self.budget_error += self.env.mu_t[self.t][arm] + self.env.mu_t[self.t][np.argmax(self.env.mu_t[self.t])] - (1.0 - self.alpha)*self.env.mu_t[self.t][self.env.safe_arm]
      #self.budget_error += (1.0 - self.alpha)*self.env.mu_t[self.t][np.argmin(self.env.safe_arm_t[self.t])] - self.env.mu_t[self.t][arm] - self.env.mu_t[self.t][np.argmax(self.env.mu_t[self.t])]


      # if self.t%20 == 0:
      #   self.detect_changepoint(self.ts, self.tp)
      

      error1 = 0
      
      
      error1 = self.env.mu_t[self.t][np.argmax(self.env.mu_t[self.t])] - self.env.mu_t[self.t][arm] + max(self.budget_error, 0.0)
      #print(self.env.mu_t[self.t][np.argmax(self.env.mu_t[self.t])], self.env.mu_t[self.t][arm])
      sum1 += error1
      self.error[tr][self.t] =  sum1
      #print(self.t,sum1)   
      
      if self.t%100 == 0 and self.t > 2.0*self.K:
        print("t " +str(self.t))
        print("t reward-arm: " + str([np.sum(self.reward_arm[int(self.ts[i]):self.t,i]) for i in range(self.K)]   ))
        #print(np.sum(self.tp), [int(self.tp[i]) for i in range(self.K)])
        #print(np.sum(self.numplays[int(np.min(self.ts)):int(np.max(self.tp)),:], axis = 1))
        #print(self.numplays[int(self.ts[2]):int(self.tp[2]),2], np.shape(self.numplays[int(self.ts[2]):int(self.tp[2]),2]))
        #print(self.numplays[int(self.ts[2]):self.t,2], np.sum(self.numplays[int(self.ts[2]):self.t,2]))
        print("t numplay: " + str([np.sum(self.numplays[int(self.ts[i]):self.t,i]) for i in range(self.K)]   ), str(np.sum(self.numplays[int(self.ts[i]):self.t,:])))
        print("t mu-hat: " + str([np.sum(self.reward_arm[int(self.ts[i]):self.t,i])/(np.sum(self.numplays[int(self.ts[i]):self.t,i]) + 1 ) for i in range(self.K)]   ))
        print("T true mu: " + str(self.env.mu_t[self.t]) + " Err: " + str(self.error[tr][self.t]))
        print(np.argmax(self.mu_hat[self.t]), np.argmax(self.env.mu_t[self.t]))
        print(self.budget, self.budget_error)
        print()
      
      self.t += 1

  def run_CUCB_sampling(self, ):

    for tr in range(0,self.trials):
      
      print(tr, end=", ")
      self.run_algorithm(tr)


# T = 8000
# trials = 10
# env = build_environment(5, 0.4, T, trials)
# env.build_env1()
cucb = CUCB_Sampling(env, 0.10, 0.1, "local")
cucb.run_CUCB_sampling()




0, t 100
t reward-arm: [2.126177444173183, 0.7263497483777211, 2.22593850989855, 2.3260361163690315, 2.008007391627463, 1.2729184690727737, 2.133914178705321, 1.2924294899525002, 2.3108220677612397, 2.099375271071498, 1.319393872276561, 0.08199514132820114, 0.06014845636064522, 1.277719701438752, 1.9359641272300583, 1.2470789934822768, 2.120148527586422, 1.291683306470683, 1.1486380711616877, 1.1122572191576263, 1.3187734275701304, 0.09011581476644095, 0.10816950328723676, 1.2592692281755145, 1.922145620298183, 1.2984792682885842, 1.3912231783890767, 1.1950868983069012, 0.9556517590902258, 2.0487303130941465, 0.9294390405762153]
t numplay: [4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 3.0, 4.0, 4.0, 3.0, 2.0, 2.0, 3.0, 4.0, 3.0, 4.0, 3.0, 3.0, 3.0, 3.0, 2.0, 2.0, 3.0, 4.0, 3.0, 3.0, 3.0, 3.0, 4.0, 3.0] 100.0
t mu-hat: [0.4252354888346366, 0.1815874370944303, 0.44518770197970997, 0.4652072232738063, 0.4016014783254926, 0.31822961726819343, 0.42678283574106424, 0.32310737248812504, 0.4621644135522

#Adversarial CUCB

In [ ]:
import numpy as np

class Adv_CUCB_Sampling(object):

  def __init__(self, env, confidence, alpha, type_):
    
    
    self.env = env
    self.confidence = confidence
    
    self.K = env.K
    self.T = env.T
    self.trials = env.trials


    self.mu_hat = np.zeros((self.T, self.K))
    self.sumreward = np.zeros((self.T, self.K))
    self.reward_arm = np.zeros((self.T, self.K))
    self.numplays = np.zeros((self.T, self.K))
    
    self.tol = 0.01
    self.gamma = 0.01
    
    self.budget = 0
    self.budget_error = 0
    self.alpha = 0.7

    self.estimated_G = np.zeros(self.K)
    self.type_ = type_
    
    
    
    self.error = np.zeros((self.trials, self.T))
  

  def detect_changepoint(self, ts, tp):

   

    for k in range(self.K):
      
      #print(tp)
      for t_ in range(int(ts[k])+1,self.t):

        
        beta_ts = np.sqrt(0.5*np.log((self.t+1)/self.confidence)/(np.sum(self.numplays[int(ts[k]):t_, k]) + 1))
        beta_tp = np.sqrt(0.5*np.log((self.t+1)/self.confidence)/(np.sum(self.numplays[t_:self.t, k]) + 1))

        if (np.sum(self.reward_arm[int(ts[k]):t_, k])/(np.sum(self.numplays[int(ts[k]):t_, k])+1) - beta_ts > np.sum(self.reward_arm[t_:self.t, k])/(np.sum(self.numplays[t_:self.t, k]) + 1) + beta_tp) \
          or (np.sum(self.reward_arm[int(ts[k]):t_, k])/(np.sum(self.numplays[int(ts[k]):t_, k]) + 1) + beta_ts < np.sum(self.reward_arm[t_:self.t, k])/(np.sum(self.numplays[t_:self.t, k]) + 1) - beta_tp): 
            
         

          if self.type_ == "global":
            for k1 in range(self.K):

              self.estimated_G[k1] += 1

              #print(self.mu_hat[int(ts[k1]): int(tp[k1]), k1])
              self.mu_hat[int(ts[k1]): self.t, k1] = 0.0
              #print(self.mu_hat[int(ts[k1]): int(tp[k1]), k1])
              self.numplays[int(ts[k1]): self.t, k1] = 0
              self.sumreward[int(ts[k1]): self.t, k1] = 0.0
              self.reward_arm[int(ts[k1]): self.t, k1] = 0.0
              self.ts[k1] = self.t

              #print(self.reward_arm[0: self.t, k])
            print("Global Changepoint Detected at time " + str(self.t))
            
            return True
          
          else:
            
            self.estimated_G[k] += 1
            self.mu_hat[int(ts[k]): self.t, k] = 0.0
            self.numplays[int(ts[k]): self.t, k] = 0
            self.sumreward[int(ts[k]): self.t, k] = 0.0
            self.reward_arm[int(ts[k]): self.t, k] = 0.0
            self.ts[k] = self.t

            

            print("Arm "+ str(k) +" Changepoint Detected at time " + str(self.t))
            continue



  def run_algorithm(self, tr):

    self.seed = tr
    np.random.seed(self.seed)
    self.t = 0
    self.ts = np.zeros(self.K)
    self.tp = np.zeros(self.K)

    self.budget = 0

    #Sample each arm once
    
    self.mu_hat = np.zeros((self.T, self.K))
    self.sumreward = np.zeros((self.T, self.K))
    self.reward_arm = np.zeros((self.T, self.K))
    self.numplays = np.zeros((self.T, self.K))

    for i in range(0,self.K):
      
      arm = i
      reward = self.env.pull_arm(arm, self.t)
      #self.sumreward[self.t][arm] += reward
      self.reward_arm[self.t][arm] = reward
      self.numplays[self.t][arm] = 1
      #self.mu_hat[self.t][arm] = self.sumreward[self.t][arm]/self.numplays[self.t][arm]

      self.tp[arm] += 1

      self.t += 1


    self.ucb = np.zeros(self.K)
    self.lcb = np.zeros(self.K)

    # self.gamma = np.sqrt((self.K*np.log(self.K)*np.log(self.T))/(self.T))
    self.gamma = 0.01
    sum1 = 0.0
    self.prob = np.ones(self.K)
    
    sum_prob = np.sum(self.prob)
    for k in range(0,self.K):
      self.prob[k] = self.prob[k]/sum_prob

    
    self.weights = np.ones(self.K)

    while self.t < self.T:
      
      for k in range(self.K):
        self.confidence = 1.0/self.confidence
        beta_ucb = np.sqrt(1.5*np.log((self.t+1)/self.confidence)/(np.sum(self.numplays[int(self.ts[k]):self.t, k]) + 1))
        self.ucb[k] = np.sum(self.reward_arm[int(self.ts[k]):self.t, k])/(np.sum(self.numplays[int(self.ts[k]):self.t, k]) + 1) + beta_ucb
        self.lcb[k] = max(0.0,np.sum(self.reward_arm[int(self.ts[k]):self.t, k])/(np.sum(self.numplays[int(self.ts[k]):self.t, k]) + 1) - beta_ucb)
        self.prob[k] = (1-self.gamma)*(self.weights[k]/sum(self.weights))+(self.gamma/self.K)


      

      sum_prob = np.sum(self.prob)
      for k in range(0,self.K):
        self.prob[k] = self.prob[k]/sum_prob

      # print(self.prob)
      if self.budget >= 0:
        arm = np.random.choice(self.K, p = self.prob)
      else:
        #arm = self.env.safe_arm
        arm = self.env.baseline

      reward = self.env.pull_arm(arm, self.t)

      xj=[0.0]*self.K

      for k in range(self.K):

        if k == arm:
          xj[k] = reward/self.prob[k]

        self.weights[k]=self.weights[k]*np.exp(self.gamma*xj[k]/self.K)
      
      #print(arm, self.budget)


      #self.sumreward[self.t][arm] += reward
      self.reward_arm[self.t][arm] = reward
      self.numplays[self.t][arm] = 1
      #self.mu_hat[self.t][arm] = self.sumreward[self.t][arm]/self.numplays[self.t][arm]
      self.mu_hat[self.t, :] = [np.sum(self.reward_arm[int(self.ts[i]):self.t,i])/(np.sum(self.numplays[int(self.ts[i]):self.t,i]) + 1 ) for i in range(self.K)]
      
      self.tp[arm] += 1
      #print("t:"+str(self.numplays[self.t]))
      # if self.t > 1.0*self.K:
      #   #print(t)

      #self.budget +=  self.lcb[arm] + self.lcb[np.argmax(self.ucb)] - (1.0-self.alpha)*self.env.mu_t[self.t][np.argmin(self.env.safe_arm_t[self.t])]
      #print(self.env.baseline)
      self.budget +=  self.lcb[arm] + self.ucb[np.argmax(self.ucb)] - (1.0-self.alpha)*self.ucb[self.env.baseline]
      #self.budget +=  self.lcb[arm] + self.lcb[np.argmax(self.ucb)] - (1.0 - self.alpha)*self.ucb[self.env.baseline]
      
      #self.budget_error += self.env.mu_t[self.t][arm] + self.env.mu_t[self.t][np.argmax(self.env.mu_t[self.t])] - (1.0 - self.alpha)*self.env.mu_t[self.t][self.env.safe_arm]
      #self.budget_error += (1.0 - self.alpha)*self.env.mu_t[self.t][np.argmin(self.env.safe_arm_t[self.t])] - self.env.mu_t[self.t][arm] - self.env.mu_t[self.t][np.argmax(self.env.mu_t[self.t])]


      # if self.t%20 == 0:
      #   self.detect_changepoint(self.ts, self.tp)
      

      error1 = 0
      
      
      error1 = self.env.mu_t[self.t][np.argmax(self.env.mu_t[self.t])] - self.env.mu_t[self.t][arm] + max(self.budget_error, 0.0)
      #print(self.env.mu_t[self.t][np.argmax(self.env.mu_t[self.t])], self.env.mu_t[self.t][arm])
      sum1 += error1
      self.error[tr][self.t] =  sum1
      # print(self.t,sum1)   
      
      if self.t%100 == 0 and self.t > 2.0*self.K:
        print("t " +str(self.t))
        print("t reward-arm: " + str([np.sum(self.reward_arm[int(self.ts[i]):self.t,i]) for i in range(self.K)]   ))
        #print(np.sum(self.tp), [int(self.tp[i]) for i in range(self.K)])
        #print(np.sum(self.numplays[int(np.min(self.ts)):int(np.max(self.tp)),:], axis = 1))
        #print(self.numplays[int(self.ts[2]):int(self.tp[2]),2], np.shape(self.numplays[int(self.ts[2]):int(self.tp[2]),2]))
        #print(self.numplays[int(self.ts[2]):self.t,2], np.sum(self.numplays[int(self.ts[2]):self.t,2]))
        print("t numplay: " + str([np.sum(self.numplays[int(self.ts[i]):self.t,i]) for i in range(self.K)]   ), str(np.sum(self.numplays[int(self.ts[i]):self.t,:])))
        print("t mu-hat: " + str([np.sum(self.reward_arm[int(self.ts[i]):self.t,i])/(np.sum(self.numplays[int(self.ts[i]):self.t,i]) + 1 ) for i in range(self.K)]   ))
        print("T true mu: " + str(self.env.mu_t[self.t]) + " Err: " + str(self.error[tr][self.t]))
        print(np.argmax(self.mu_hat[self.t]), np.argmax(self.env.mu_t[self.t]))
        print(self.budget, self.budget_error)
        print(self.prob)
        print()
      
      self.t += 1

  def run_Adv_CUCB_sampling(self, ):

    for tr in range(0,self.trials):
      
      print(tr, end=", ")
      self.run_algorithm(tr)


# T = 8000
# trials = 10
# env = build_environment(5, 0.4, T, trials)
# env.build_env1()
adv_cucb = Adv_CUCB_Sampling(env, 0.10, 0.1, "local")
adv_cucb.run_Adv_CUCB_sampling()




Streaming output truncated to the last 5000 lines.
619.0726630353348 0
[0.03238258 0.03134148 0.03233552 0.03360261 0.03307902 0.03194126
 0.03227493 0.03291906 0.0330231  0.03277192 0.03269299 0.03070122
 0.030833   0.0333243  0.03285385 0.03254676 0.03290039 0.03180291
 0.03205439 0.03219015 0.03273858 0.03088389 0.03100468 0.03228112
 0.0314686  0.03169695 0.03212083 0.03234142 0.03233711 0.03366962
 0.03188576]

t 500
t reward-arm: [8.335485489975252, 2.9907236195703852, 5.89972105102362, 14.156649098056084, 8.249828098228477, 5.050910179265873, 6.251871004473564, 10.012637894404651, 8.544217948380496, 9.111993862785853, 9.954944735718726, -0.19124526001221548, 0.3056116341524382, 9.457290082852357, 10.068119134693747, 7.641584384088283, 7.640178690984689, 4.282658733690731, 4.716837188876687, 5.43415256192889, 7.335325796352757, 0.17368411166833592, 0.382119491021695, 5.514714110556413, 4.014390111143372, 4.849369314605086, 5.03017875009696, 5.664013224407073, 6.7125505736066575, 

# SLR 

In [ ]:
import numpy as np
from multiprocessing import Process, Queue
import time

class SLR_Sampling(object):

  def __init__(self, env, confidence, alpha, type_):
    
    
    self.env = env
    self.confidence = confidence
    
    self.K = env.K
    self.T = env.T
    self.trials = env.trials


    self.mu_hat = np.zeros((self.T, self.K))
    self.sumreward = np.zeros((self.T, self.K))
    self.reward_arm = np.zeros((self.T, self.K))
    self.numplays = np.zeros((self.T, self.K))
    
    self.tol = 0.01
    self.gamma = 0.01
    
    self.budget = 0
    self.budget_error = 0
    self.alpha = 0.7

    self.estimated_G = np.zeros(self.K)
    self.type_ = type_
    
    
    
    self.error = np.zeros((self.trials, self.T))
  

  def detect_changepoint(self, ts, tp):

   

    for k in range(self.K):
      
      #print(tp)
      for t_ in range(int(ts[k])+1,self.t):

        
        beta_ts = np.sqrt(0.5*np.log(np.log(self.t+1)/self.confidence)/(np.sum(self.numplays[int(ts[k]):t_, k]) + 1))
        beta_tp = np.sqrt(0.5*np.log(np.log(self.t+1)/self.confidence)/(np.sum(self.numplays[t_:self.t, k]) + 1))

        if (np.sum(self.reward_arm[int(ts[k]):t_, k])/(np.sum(self.numplays[int(ts[k]):t_, k])+1) - beta_ts > np.sum(self.reward_arm[t_:self.t, k])/(np.sum(self.numplays[t_:self.t, k]) + 1) + beta_tp) \
          or (np.sum(self.reward_arm[int(ts[k]):t_, k])/(np.sum(self.numplays[int(ts[k]):t_, k]) + 1) + beta_ts < np.sum(self.reward_arm[t_:self.t, k])/(np.sum(self.numplays[t_:self.t, k]) + 1) - beta_tp): 
            
         

          if self.type_ == "global":
            for k1 in range(self.K):

              self.estimated_G[k1] += 1

              #print(self.mu_hat[int(ts[k1]): int(tp[k1]), k1])
              self.mu_hat[int(ts[k1]): self.t, k1] = 0.0
              #print(self.mu_hat[int(ts[k1]): int(tp[k1]), k1])
              self.numplays[int(ts[k1]): self.t, k1] = 0
              self.sumreward[int(ts[k1]): self.t, k1] = 0.0
              self.reward_arm[int(ts[k1]): self.t, k1] = 0.0
              self.ts[k1] = self.t
              self.safe_set = set()
              self.budget = 0.0
              #print(self.reward_arm[0: self.t, k])
            print("Global Changepoint Detected at time " + str(self.t))
            
            return True
          
          else:
            
            self.estimated_G[k] += 1
            self.mu_hat[int(ts[k]): self.t, k] = 0.0
            self.numplays[int(ts[k]): self.t, k] = 0
            self.sumreward[int(ts[k]): self.t, k] = 0.0
            self.reward_arm[int(ts[k]): self.t, k] = 0.0
            self.ts[k] = self.t
            self.safe_set = set()
            self.budget = 0.0

            print("Arm "+ str(k) +" Changepoint Detected at time " + str(self.t))
            continue

  def safe_set_function(self, ):
    
    self.safe_set = set()
    for k in range(0, self.K-1):

      #mu_hat = np.sum(self.reward_arm[int(self.ts[k]):self.t, k])/(np.sum(self.numplays[int(self.ts[k]):self.t, k]) + 1)
      #if self.lcb[k] > self.env.threshold and self.ucb[k] < 1.5*self.env.threshold:
      #if self.lcb[k] > self.env.threshold:
      #if self.mu_hat[self.t-1][k] > self.env.threshold and self.mu_hat[self.t-1][k]  < 1.5*self.env.threshold:
      if self.mu_hat[self.t-1][k] >= self.alpha*self.env.threshold:
        
        self.safe_set.add((k, self.mu_hat[self.t-1][k]))
    
    return self.safe_set

  def run_algorithm(self, tr):

    self.seed = tr
    np.random.seed(self.seed)
    self.t = 0
    self.ts = np.zeros(self.K)
    self.tp = np.zeros(self.K)

    self.budget = 0

    self.mu_hat = np.zeros((self.T, self.K))
    self.sumreward = np.zeros((self.T, self.K))
    self.reward_arm = np.zeros((self.T, self.K))
    self.numplays = np.zeros((self.T, self.K))

    #Sample each arm once
    

    for i in range(0,self.K):
      
      arm = i
      reward = self.env.pull_arm(arm, self.t)
      #self.sumreward[self.t][arm] += reward
      self.reward_arm[self.t][arm] = reward
      self.numplays[self.t][arm] = 1
      #self.mu_hat[self.t][arm] = self.sumreward[self.t][arm]/self.numplays[self.t][arm]

      self.tp[arm] += 1

      self.t += 1



    sum1 = 0.0
    self.ucb = np.zeros(self.K)
    self.lcb = np.zeros(self.K)
    self.B = np.zeros(self.K)
    self.safe_set = set()
    self.gamma = np.sqrt(np.log(self.T)/self.T)

    self.error_tr = np.zeros(self.T)

    while self.t < self.T:
      
      for k in range(self.K):

        beta_ucb = np.sqrt(1.0*np.log(np.log(self.t+1)/self.confidence)/(np.sum(self.numplays[int(self.ts[k]):self.t, k]) + 1))
        self.ucb[k] = np.sum(self.reward_arm[int(self.ts[k]):self.t, k])/(np.sum(self.numplays[int(self.ts[k]):self.t, k]) + 1) + beta_ucb
        self.lcb[k] = max(0.0,np.sum(self.reward_arm[int(self.ts[k]):self.t, k])/(np.sum(self.numplays[int(self.ts[k]):self.t, k]) + 1) - beta_ucb)
        self.B[k] = abs(np.sum(self.reward_arm[int(self.ts[k]):self.t, k])/(np.sum(self.numplays[int(self.ts[k]):self.t, k]) + 1) - self.alpha*self.env.threshold)*np.sqrt((np.sum(self.numplays[int(self.ts[k]):self.t, k]) + 1))

      
      self.safe_set = self.safe_set_function()

      if self.budget >= 0:
        if self.t% (self.K/self.gamma) <= self.K:
          arm = np.random.randint(0, self.K-1)
          #print("fe", arm, self.t% (self.K/self.gamma))
        else:
          arm = np.argmax(self.ucb)
      elif self.budget < 0:
        arm = self.env.baseline

      

      
      reward = self.env.pull_arm(arm, self.t)
      #self.sumreward[self.t][arm] += reward
      self.reward_arm[self.t][arm] = reward
      self.numplays[self.t][arm] = 1
      #self.mu_hat[self.t][arm] = self.sumreward[self.t][arm]/self.numplays[self.t][arm]
      self.mu_hat[self.t, :] = [np.sum(self.reward_arm[int(self.ts[i]):self.t,i])/(np.sum(self.numplays[int(self.ts[i]):self.t,i]) + 1 ) for i in range(self.K)]
      
      self.tp[arm] += 1
      #print("t:"+str(self.numplays[self.t]))
      # if self.t > 1.0*self.K:
      #   #print(t)

      # self.budget +=  self.lcb[arm] + self.lcb[np.argmax(self.ucb)] - (1.0 - self.alpha)*self.ucb[self.env.safe_arm]
      # self.budget_error += (1.0 - self.alpha)*self.env.mu_t[self.t][self.env.safe_arm] - self.env.mu_t[self.t][arm] - self.env.mu_t[self.t][np.argmax(self.env.mu_t[self.t])]

      # self.budget +=  self.lcb[arm] + self.lcb[np.argmax(self.ucb)] - (1.0 - self.alpha)*self.ucb[np.argmin(self.safe_set)]
      # self.budget_error += (1.0 - self.alpha)*self.env.mu_t[self.t][self.env.safe_arm] - self.env.mu_t[self.t][arm] - self.env.mu_t[self.t][np.argmin(self.env.mu_t[self.t])]

      # if len(self.safe_set) > 0:
      #   self.budget +=  self.lcb[arm] + self.lcb[np.argmax(self.ucb)] - (1.0 - self.alpha)*self.ucb[min(self.safe_set, key = lambda x: x[1])[0]]
      self.budget +=  self.lcb[arm] + self.ucb[np.argmax(self.mu_hat[self.t, :])] - (1.0 - self.alpha)*self.ucb[self.env.baseline]
      #self.budget_error += self.env.mu_t[self.t][arm] + self.env.mu_t[self.t][np.argmax(self.env.mu_t[self.t])] - (1.0 - self.alpha)*self.env.mu_t[self.t][self.env.safe_arm]
      #self.budget_error += (1.0 - self.alpha)*self.env.mu_t[self.t][np.argmin(self.env.safe_arm_t[self.t])] - self.env.mu_t[self.t][arm] - self.env.mu_t[self.t][np.argmin(self.env.mu_t[self.t])]


      if self.t%100 == 0:
        self.detect_changepoint(self.ts, self.tp)
        
      error1 = 0
      
      
      error1 = self.env.mu_t[self.t][np.argmax(self.env.mu_t[self.t])] - self.env.mu_t[self.t][arm] + max(self.budget_error, 0.0)
      #print(self.env.mu_t[self.t][np.argmax(self.env.mu_t[self.t])], self.env.mu_t[self.t][arm])
      sum1 += error1
      # self.error[tr][self.t] =  sum1
      self.error_tr[self.t] = sum1
      #print(self.t,sum1)   
      
      if self.t%100 == 0 and self.t > 2.0*self.K:
        print("t " +str(self.t))
        print("t reward-arm: " + str([np.sum(self.reward_arm[int(self.ts[i]):self.t,i]) for i in range(self.K)]   ))
        #print(np.sum(self.tp), [int(self.tp[i]) for i in range(self.K)])
        #print(np.sum(self.numplays[int(np.min(self.ts)):int(np.max(self.tp)),:], axis = 1))
        #print(self.numplays[int(self.ts[2]):int(self.tp[2]),2], np.shape(self.numplays[int(self.ts[2]):int(self.tp[2]),2]))
        #print(self.numplays[int(self.ts[2]):self.t,2], np.sum(self.numplays[int(self.ts[2]):self.t,2]))
        print("t numplay: " + str([np.sum(self.numplays[int(self.ts[i]):self.t,i]) for i in range(self.K)]   ), str(np.sum(self.numplays[int(self.ts[i]):self.t,:])))
        print("t mu-hat: " + str([np.sum(self.reward_arm[int(self.ts[i]):self.t,i])/(np.sum(self.numplays[int(self.ts[i]):self.t,i]) + 1 ) for i in range(self.K)]   ))
        print("T true mu: " + str(self.env.mu_t[self.t]) + " Err: " + str(self.error_tr[self.t]))
        print(np.argmax(self.mu_hat[self.t]), np.argmax(self.env.mu_t[self.t]))
        print(self.budget, self.budget_error, self.safe_set)
        print()
      
      self.t += 1

    file = open("/content/f_SLR_"+str(tr), 'wb')
    data = pickle.dump(self.error_tr,file)
    file.close()

  # def run_SLR_sampling(self, ):

  #   for tr in range(0,self.trials):
      
  #     print(tr, end=", ")
  #     self.run_algorithm(tr)
  

  def run_SLR_sampling(self):

        
      self.error = np.zeros((self.trials, self.T))  
      pool = []
      Q = Queue()

      for tr in range(self.trials):
        
        p = Process(target = self.run_algorithm, args=(tr, ))
        pool.append(p)
          
      for tr in range(self.trials):
        pool[tr].start()

      

      for tr in range(self.trials):
        # self.error[tr] = Q.get()
        pool[tr].join()
      
      for tr in range(self.trials):
        file = open("/content/f_SLR_"+str(tr), 'rb')
        self.error[tr] = pickle.load(file)
        file.close()

      for tr in range(self.trials):
        pool[tr].close()



# T = 8000
# trials = 10
# env = build_environment(5, 0.4, T, trials)
# env.build_env1()
slr = SLR_Sampling(env, 0.10, 0.1, "local")
slr.run_SLR_sampling()




t 100
t reward-arm: [2.1218184011366232, 0.5187812395014364, 2.280244718579417, 2.2101501835533686, 2.2164563728017055, 1.3342599047864132, 1.375377329035682, 1.301912517685055, 1.4281489514439125, 2.176409188249292, 2.193789096836124, 0.10340901172942468, -0.07586346534852716, 2.2571684410348882, 2.0309491343427784, 1.9930287353276355, 2.1389773272302253, 1.4328591369226322, 1.3695805450923126, 1.3965354146214826, 0.5311181778119329, 0.17676257868843387, 0.021380122965109785, 1.331337398761149, 2.0759897394911664, 1.0790544339765853, 1.9872153656516045, 1.0574526192085605, 1.208537480536901, 1.972565636132415, 0.9582074661451999]
t numplay: [4.0, 2.0, 4.0, 4.0, 4.0, 3.0, 3.0, 3.0, 3.0, 4.0, 4.0, 2.0, 2.0, 4.0, 4.0, 4.0, 4.0, 3.0, 3.0, 3.0, 2.0, 2.0, 2.0, 3.0, 4.0, 3.0, 4.0, 3.0, 3.0, 4.0, 3.0] 100.0
t 100
t mu-hat: [0.4243636802273246, 0.17292707983381214, 0.4560489437158834, 0.4420300367106737, 0.4432912745603411, 0.3335649761966033, 0.3438443322589205, 0.32547812942126375, 0.3570372

# File Save

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
file = open('/content/drive/My Drive/Expts/SafeChangepoint/Local_Movielens_Data6', 'wb')

A = umoss.error
B = ucb.error
C = cucb.error
D = glrucb.error
E = slr.error
F = ducb.error
G = adv_cucb.error

pickle.dump([A, B, C, D, E, F, G],file) # dump information to that file

# print(adv_cucb.error)
file.close()

# Load Data

In [ ]:
import pickle 
file = open('/content/drive/My Drive/Expts/SafeChangepoint/Local_Movielens_Data6', 'rb')
data = pickle.load(file)
file.close()

# umoss = UMOSS_Sampling(env, 0.10, 0.1, "local")
# ucb = UCB_Sampling(env, 0.10, "local")
# cucb = CUCB_Sampling(env, 0.10, 0.1, "local")
# glrucb = GLR_UCB_Sampling(env, 0.10, "local")
# slr = SLR_Sampling(env, 0.10, 0.1, "local")


umoss.error = data[0]
ucb.error = data[1]
cucb.error = data[2]
glrucb.error = data[3]
slr.error = data[4]
ducb.error = data[5]
adv_cucb.error = data[6]

print(data[6])

[[   0.       0.       0.    ... 1169.183 1169.256 1169.263]
 [   0.       0.       0.    ... 1160.168 1160.261 1160.34 ]
 [   0.       0.       0.    ... 1175.074 1175.16  1175.195]
 [   0.       0.       0.    ... 1166.972 1167.487 1167.54 ]]


# Regret Plot

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

plt.style.use('ggplot')
plt.figure(dpi=200)
k = 1.0

plt.plot(np.mean(umoss.error, axis = 0), label = "UMOSS", color = (0.78,0.4,0.4))
plt.fill_between(range(env.T), np.average(umoss.error, axis=0) + k*np.std(umoss.error, axis=0), np.average(umoss.error, axis=0) - k*np.std(umoss.error, axis=0), alpha=0.4, edgecolor=(0.78,0.4,0.4), facecolor=(0.78,0.4,0.4), linewidth=2, linestyle='dashdot', antialiased=True)


plt.plot(np.mean(ucb.error, axis = 0), label = "UCB-CPD-E", color = (0.2,0.7,0.63))
plt.fill_between(range(env.T), np.average(ucb.error, axis=0) + k*np.std(ucb.error, axis=0), np.average(ucb.error, axis=0) - k*np.std(ucb.error, axis=0), alpha=0.4, edgecolor=(0.2,0.7,0.63), facecolor=(0.2,0.7,0.63), linewidth=2, linestyle='dashdot', antialiased=True)

plt.plot(np.mean(cucb.error, axis = 0), label = "CUCB", color = (0.3,0.2,0.9))
plt.fill_between(range(env.T), np.average(cucb.error, axis=0) + k*np.std(cucb.error, axis=0), np.average(cucb.error, axis=0) - k*np.std(cucb.error, axis=0), alpha=0.4, edgecolor=(0.3,0.2,0.9), facecolor=(0.3,0.2,0.9), linewidth=2, linestyle='dashdot', antialiased=True)

plt.plot(np.mean(adv_cucb.error, axis = 0), label = "Adv-CUCB", color = (0.9,0.2,0.9))
plt.fill_between(range(env.T), np.average(adv_cucb.error, axis=0) + k*np.std(adv_cucb.error, axis=0), np.average(adv_cucb.error, axis=0) - k*np.std(adv_cucb.error, axis=0), alpha=0.4, edgecolor=(0.9,0.2,0.9), facecolor=(0.9,0.2,0.9), linewidth=2, linestyle='dashdot', antialiased=True)


plt.plot(np.mean(ducb.error, axis = 0), label = "DUCB", color = (0.4,0.6,0.21))
plt.fill_between(range(env.T), np.average(ducb.error, axis=0) + k*np.std(ducb.error, axis=0), np.average(ducb.error, axis=0) - k*np.std(ducb.error, axis=0), alpha=0.4, edgecolor=(0.4,0.6,0.21), facecolor=(0.4,0.6,0.21), linewidth=2, linestyle='dashdot', antialiased=True)

plt.plot(np.mean(glrucb.error, axis = 0), label = "GLR-UCB", color = (0.6,0.2,0.96))
plt.fill_between(range(env.T), np.average(glrucb.error, axis=0) + k*np.std(glrucb.error, axis=0), np.average(glrucb.error, axis=0) - k*np.std(glrucb.error, axis=0), alpha=0.4, edgecolor=(0.6,0.2,0.96), facecolor=(0.6,0.2,0.96), linewidth=2, linestyle='dashdot', antialiased=True)


plt.plot(np.mean(slr.error, axis = 0), label = "SLR", color = "r")
plt.fill_between(range(env.T), np.average(slr.error, axis=0) + k*np.std(slr.error, axis=0), np.average(slr.error, axis=0) - k*np.std(slr.error, axis=0), alpha=0.4, edgecolor="r", facecolor="r", linewidth=2, linestyle='dashdot', antialiased=True)


plt.title('Local Changepoint Setting (Movielens)', size = 15, fontweight='bold')
plt.ylabel('Cumulative Regret', size = 15, fontweight='bold')
plt.xlabel('Rounds', size = 15, fontweight='bold')
    
plt.yticks(size = 15, fontweight='bold')
plt.xticks(size = 15, fontweight='bold')

plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
plt.ticklabel_format(axis='x', style='sci', scilimits=(0,0))


plt.xlim(env.K)
plt.grid()
plt.legend()
# plt.ylim(0,800)


plt.show()
